### Import Libraries

In [ ]:
import os
import pathlib
import numpy as np

from PIL import Image
from glob import glob

import tensorflow as tf
import matplotlib.pyplot as plt

### Get image files

In [ ]:
# Deploy the flower datasets which consists of 3,700 files

dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)

data_dir

In [ ]:
# daisy 폴더 안의 이미지 갯수
!ls -l /home/jihwan/.keras/datasets/flower_photos/daisy | grep jpg | wc -l
# dandelion 폴더 안의 이미지 갯수
!ls -l /home/jihwan/.keras/datasets/flower_photos/dandelion | grep jpg | wc -l
# roses 폴더 안의 이미지 갯수
!ls -l /home/jihwan/.keras/datasets/flower_photos/roses | grep jpg | wc -l
# sunflowers 폴더 안의 이미지 갯수
!ls -l /home/jihwan/.keras/datasets/flower_photos/sunflowers | grep jpg | wc -l
# tulips 폴더 안의 이미지 갯수
!ls -l /home/jihwan/.keras/datasets/flower_photos/tulips | grep jpg | wc -l

### Read Image using listdir function and PIL.Image

In [ ]:
daisy_path = f'{str(data_dir)}/daisy'
daisy_file = os.listdir(daisy_path)

dandelion_path = f'{str(data_dir)}/dandelion'
dandelion_file = os.listdir(dandelion_path)

roses_path = f'{str(data_dir)}/roses'
roses_file = os.listdir(roses_path)

sunflowers_path = f'{str(data_dir)}/sunflowers'
sunflowers_file = os.listdir(sunflowers_path)

tulips_path = f'{str(data_dir)}/tulips'
tulips_file = os.listdir(tulips_path)

In [ ]:
for img_file in daisy_file[:2]:
    img = Image.open(os.path.join(daisy_path, img_file)).resize((224,224))
    plt.title(img_file + ' : Positive')
    plt.imshow(img)
    plt.show()

for img_file in roses_file[:2]:
    img = Image.open(os.path.join(roses_path, img_file)).resize((224,224))
    plt.title(img_file + ' : Negative')
    plt.imshow(img)
    plt.show()

### Make List of image including image labeling

In [ ]:
# self-list of image and label
class2idx = {
  'daisy' :  0, 
  'dandelion' : 1,
  'roses' : 2,
  'sunflowers' : 3,
  'tulips' : 4,
}
idx2class = {
  0 : 'daisy',
  1 : 'dandelion',
  2 : 'roses',
  3 : 'sunflowers',
  4 : 'tulips',
}

img_list, label_list = [], []

for idx, filepath in enumerate((daisy_path, dandelion_path, roses_path, sunflowers_path, tulips_path, )):
  for img_file in os.listdir(filepath):
    img = Image.open(os.path.join(filepath , img_file)).resize((128,128)) # (128, 128, 3)
    img = np.array(img) / 255.0
    img_list.append(img)
    label_list.append(idx)

In [ ]:
img_list_arr = np.array(img_list)
label_list_arr = np.array(label_list)

img_list_arr.shape, label_list_arr.shape

### 5. Make Train / Test Dataset

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    img_list_arr, 
    label_list_arr,
    test_size=0.3,
    stratify=label_list_arr,
    random_state=41,
)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

### Build CNN Model using TensorFlow

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D,
    Dense, Dropout, Flatten, 
)

# Hyperparameter Tunning
input_shape = (128, 128, 3)
num_epochs = 10
batch_size = 32
learning_rate = 0.001
dropout_rate = 0.5

model = Sequential()

# Setting Convolution Model
model.add(Conv2D(32, (5,5), activation='relu', padding='same', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Conv2D(64, (2,2), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(5, activation='softmax')) # num. of labels = 5

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
    loss='sparse_categorical_crossentropy',  # Loss Function
    metrics=['accuracy'],  # Metrics / Accuracy
)
model.summary()

### Callback

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

checkpoint_path = "my_checkpoint.keras"
checkpoint = ModelCheckpoint(
    filepath=checkpoint_path,
    save_best_only=True,
    monitor='val_loss',
    verbose=1,
)

### Training Model

In [ ]:
history = model.fit(
    X_train, y_train ,
    validation_data=(X_test, y_test),
    epochs=num_epochs,
    batch_size=batch_size,
    callbacks=[es, checkpoint]
)

### Performance Graph

In [ ]:
history.history.keys()

In [ ]:
plt.plot(history.history['accuracy'], label='Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Model Accuracy')
plt.show()

### Prediction

In [ ]:
plt.figure(figsize=(16, 8))

i = 1
for img, label in zip(X_test[:8], y_test[:8]):
      pred = model.predict(img.reshape(-1, 128, 128, 3))
      pred_t = np.argmax(pred)
      plt.subplot(2, 4, i)
      plt.title(f'True Value:{label}, Pred Value: {pred_t}')
      plt.imshow(img)
      plt.axis('off')
      i += 1